In [ ]:
import sqlite3

# connect (creates file if it doesn't exist)
conn = sqlite3.connect("university.db")
cur = conn.cursor()

# make sure FK constraints are enforced
cur.execute("PRAGMA foreign_keys = ON;")

# ----- base tables -----

cur.execute("""
CREATE TABLE department (
    dept_name   TEXT PRIMARY KEY,
    building    TEXT,
    budget      NUMERIC
);
""")

cur.execute("""
CREATE TABLE classroom (
    building    TEXT,
    room_number TEXT,
    capacity    INTEGER,
    PRIMARY KEY (building, room_number)
);
""")

cur.execute("""
CREATE TABLE time_slot (
    time_slot_id TEXT PRIMARY KEY,
    day          TEXT,
    start_hr     INTEGER,
    start_min    INTEGER,
    end_hr       INTEGER,
    end_min      INTEGER
);
""")

cur.execute("""
CREATE TABLE course (
    course_id  TEXT PRIMARY KEY,
    title      TEXT,
    dept_name  TEXT,
    credits    INTEGER,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

cur.execute("""
CREATE TABLE student (
    ID        TEXT PRIMARY KEY,
    name      TEXT,
    dept_name TEXT,
    tot_cred  INTEGER,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

cur.execute("""
CREATE TABLE instructor (
    ID        TEXT PRIMARY KEY,
    name      TEXT,
    dept_name TEXT,
    salary    NUMERIC,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

# ----- relationship tables -----

cur.execute("""
CREATE TABLE advisor (
    s_ID  TEXT PRIMARY KEY,
    i_ID  TEXT,
    FOREIGN KEY (s_ID) REFERENCES student(ID),
    FOREIGN KEY (i_ID) REFERENCES instructor(ID)
);
""")

cur.execute("""
CREATE TABLE section (
    course_id   TEXT,
    sec_id      TEXT,
    semester    TEXT,
    year        INTEGER,
    building    TEXT,
    room_number TEXT,
    time_slot_id TEXT,
    PRIMARY KEY (course_id, sec_id, semester, year),
    FOREIGN KEY (course_id) REFERENCES course(course_id),
    FOREIGN KEY (building, room_number)
        REFERENCES classroom(building, room_number),
    FOREIGN KEY (time_slot_id) REFERENCES time_slot(time_slot_id)
);
""")

cur.execute("""
CREATE TABLE takes (
    ID        TEXT,
    course_id TEXT,
    sec_id    TEXT,
    semester  TEXT,
    year      INTEGER,
    grade     TEXT,
    PRIMARY KEY (ID, course_id, sec_id, semester, year),
    FOREIGN KEY (ID) REFERENCES student(ID),
    FOREIGN KEY (course_id, sec_id, semester, year)
        REFERENCES section(course_id, sec_id, semester, year)
);
""")

cur.execute("""
CREATE TABLE teaches (
    ID        TEXT,
    course_id TEXT,
    sec_id    TEXT,
    semester  TEXT,
    year      INTEGER,
    PRIMARY KEY (ID, course_id, sec_id, semester, year),
    FOREIGN KEY (ID) REFERENCES instructor(ID),
    FOREIGN KEY (course_id, sec_id, semester, year)
        REFERENCES section(course_id, sec_id, semester, year)
);
""")

cur.execute("""
CREATE TABLE prereq (
    course_id  TEXT,
    prereq_id  TEXT,
    PRIMARY KEY (course_id, prereq_id),
    FOREIGN KEY (course_id) REFERENCES course(course_id),
    FOREIGN KEY (prereq_id) REFERENCES course(course_id)
);
""")

conn.commit()
conn.close()
